<a href="https://colab.research.google.com/github/mgmills93/AEM_Spring_2019/blob/master/SecureCommunities_FinalProject/FinalProject_Hispanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Question 4:	Is the number of state SC arrests correlated with the state’s Hispanic population?
---
I will compare SC arrests by state from 2015-2017 with the percentage of that state's population that claimed some hispanic origin in 2016. 

This notebook will be updated with a pipeline of commands; I have been having trouble with the file so this one is a placeholder that arrives at the same dataset. (4/21/19)

In [1]:
import pandas as pd
import numpy as np
sc_arrests = pd.read_excel('https://raw.githubusercontent.com/mgmills93/AEM_Spring_2019/master/enforcement/Secure_Communities_2017_Deportations.xlsx', header=4)
print(sc_arrests.head(5))


  State    County Departed Date             Port Of Departure  \
0    TX    Howard    2016-03-28              DEL RIO, TX, POE   
1    TX    Harris    2015-12-09               LAREDO, TX, POE   
2    IN    Marion    2015-03-06  BROWNSVILLE-MATAMOR, TX, POE   
3    TX    Reeves    2016-01-13              EL PASO, TX, POE   
4    AZ  Maricopa    2015-12-19              NOGALES, AZ, POE   

  Departed To Country                           Case Status  \
0              MEXICO  8-Excluded/Removed - Inadmissibility   
1              MEXICO  8-Excluded/Removed - Inadmissibility   
2              MEXICO    6-Deported/Removed - Deportability   
3            HONDURAS    6-Deported/Removed - Deportability   
4              MEXICO  8-Excluded/Removed - Inadmissibility   

                                       Case Category  \
0                        [16] Reinstated Final Order   
1  [8C] Excludable / Inadmissible - Administrativ...   
2          [11] Administrative Deportation / Removal   
3     

In [0]:
county_state = sc_arrests.loc[:, 'State':'Departed Date']

In [4]:
cs_group = county_state.groupby(['State', 'County']).count().reset_index()
cs_group.rename(columns={'Departed Date':'Arrest_Number'}, inplace=True)
cs_group['County_State'] = cs_group['County'].map(str) + ', ' + cs_group['State']
print(cs_group)

     State                     County  Arrest_Number  \
0       AK          Anchorage Borough              9   
1       AK  Matanuska-Susitna Borough              1   
2       AL                    Autauga              4   
3       AL                    Baldwin            145   
4       AL                    Barbour              2   
5       AL                     Blount              4   
6       AL                    Calhoun              7   
7       AL                   Cherokee              1   
8       AL                    Chilton             25   
9       AL                       Clay              1   
10      AL                   Cleburne              3   
11      AL                     Coffee              6   
12      AL                    Colbert              3   
13      AL                   Crenshaw              1   
14      AL                    Cullman             11   
15      AL                       Dale              3   
16      AL                     DeKalb           

In [6]:
hisp_counties = pd.read_csv('https://raw.githubusercontent.com/mgmills93/AEM_Spring_2019/master/SecureCommunities_FinalProject/Datasets/ACS_16_5YR_B03001_with_ann_copy.csv', encoding='latin-1', header=1)
geo_pop = hisp_counties.loc[:, 'Geography':'Estimate; Total: - Hispanic or Latino:']
new = geo_pop['Geography'].str.split(', ', n=1, expand=True)
geo_pop['County']=new[0]
geo_pop['State']=new[1]
geo_pop.drop(columns=['Geography'], inplace=True)

geo_pop['County'] = geo_pop['County'].map(lambda x: x.rstrip('County'))

geo_pop['Percent_Hispanic'] = geo_pop['Estimate; Total: - Hispanic or Latino:']/geo_pop['Estimate; Total:']*100
geo_pop.drop(columns=['Estimate; Total:', 'Estimate; Total: - Hispanic or Latino:'], inplace=True)

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Colmbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',}

geo_pop['State']=geo_pop['State'].map(us_state_abbrev)
geo_pop = geo_pop.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
geo_pop['County_State'] = geo_pop['County'].map(str) + ', ' + geo_pop['State']

print(geo_pop.head(10))


     County State  Percent_Hispanic  County_State
0   Autauga    AL          2.572254   Autauga, AL
1   Baldwin    AL          4.366698   Baldwin, AL
2   Barbour    AL          4.309762   Barbour, AL
3      Bibb    AL          2.223994      Bibb, AL
4    Blount    AL          8.727298    Blount, AL
5   Bullock    AL          0.123199   Bullock, AL
6    Butler    AL          1.247535    Butler, AL
7   Calhoun    AL          3.436224   Calhoun, AL
8  Chambers    AL          0.435064  Chambers, AL
9  Cherokee    AL          1.563888  Cherokee, AL


In [11]:
final_hisp_set = cs_group.merge(geo_pop, on="County_State", how="inner")
final_hisp_set = final_hisp_set.loc[:, ['Arrest_Number', 'County_State', 'Percent_Hispanic']]
print(final_hisp_set.tail(10))
final_hisp_set.to_csv('final_hisp_set.csv')

      Arrest_Number    County_State  Percent_Hispanic
2033             36     Laramie, WY         14.199815
2034              2     Lincoln, WY          4.416761
2035             19     Natrona, WY          8.198242
2036              4        Park, WY          5.618403
2037              4      Platte, WY          7.814645
2038              3    Sheridan, WY          4.113755
2039              1    Sublette, WY          7.814992
2040             26  Sweetwater, WY         15.859591
2041             32       Teton, WY         15.174822
2042              3       Uinta, WY          8.959939
